In [1]:
# # # Parameters
DT = "2024-06-11"   

## **One Model Profile 생성 작업**

- 개요
    - One Model Profile 생성 작업
- 내용
    - item, cat1, cat2 level로 프로파일링 작업 수행
    - 단기 preference 정보
        - 7일 내 item 정보를 통해서 생성
    - 장기 preference 정보
        - 60일 내 cat1, cat2 정보를 통해서 생성
        - cat2가 더 상위 레벨임. 어떤 레벨로 장기 구성할지 선택 필요

In [2]:
from skt.gcp import get_bigquery_client, bq_insert_overwrite, get_max_part, bq_to_df, bq_to_pandas, pandas_to_bq_table, load_query_result_to_table, df_to_bq_table
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql import functions as F
from datetime import datetime, date, timedelta
from skt.ye import get_spark
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# 프로파일 : xtr

In [3]:
# DT = "2024-06-02"
current_date = datetime.strptime(DT, "%Y-%m-%d")
DT_threshold1 = (current_date - timedelta(days=60)).strftime("%Y-%m-%d")
DT_threshold2 = (current_date - timedelta(days=7)).strftime("%Y-%m-%d")

print("DT : ", DT)
print("DT_threshold : ", DT_threshold1)
print("DT_threshold : ", DT_threshold2)

DT :  2024-06-11
DT_threshold :  2024-04-12
DT_threshold :  2024-06-04


In [4]:
query = f"""
    select *
    from adot_reco.recgpt_log_sequence_daily_prd
    where dt >= '{DT_threshold1}' and type = 'xtr'
"""

In [5]:
data = bq_to_df(query)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/12 08:43:01 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.
24/06/12 08:43:23 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.


In [8]:
# data.count()

In [9]:
data.show(n=10)

+--------------------+----+-------------+-----------------+----+--------------------+--------+----------+----+----------+----+
|        svc_mgmt_num|item|         cat1|             cat2|cat3|           unix_time|second_s|      etc1|etc2|        dt|type|
+--------------------+----+-------------+-----------------+----+--------------------+--------+----------+----+----------+----+
|560181909b70857c4...|  5G|모바일 요금제|티월드 모바일 APP|null|1712827263.000000000|   33663|상품서비스|null|2024-04-11| xtr|
|b283368c4b237be00...|  5G|모바일 요금제|티월드 모바일 APP|null|1712824112.000000000|   30512|상품서비스|null|2024-04-11| xtr|
|5fa4ad46c41a6c692...|  5G|모바일 요금제|티월드 모바일 APP|null|1712806155.000000000|   12555|상품서비스|null|2024-04-11| xtr|
|bc1da8929dfe50baf...|  5G|모바일 요금제|티월드 모바일 APP|null|1712801735.000000000|    8135|상품서비스|null|2024-04-11| xtr|
|5c899a210f4be221c...|  5G|모바일 요금제|티월드 모바일 APP|null|1712811742.000000000|   18142|상품서비스|null|2024-04-11| xtr|
|3b9b210e3930e5a06...|  5G|모바일 요금제|티월드 모바일 APP|null|1712826575.000000

In [10]:
data.select('cat3').distinct().show()

+----+
|cat3|
+----+
|null|
+----+



In [11]:
data.select('cat2').distinct().show()

+-----------------+
|             cat2|
+-----------------+
|티월드 모바일 APP|
|    티월드 온라인|
|티월드 모바일 WEB|
+-----------------+



In [12]:
data.select('cat1').distinct().show()

+-------------------------+
|                     cat1|
+-------------------------+
|                 결합상품|
|            회원정보 관리|
|            모바일 요금제|
|            자주하는 질문|
|                로밍 상품|
|                 본인인증|
|                     인증|
|                고객 의견|
|          회원가입/로그인|
|   신규기변 통합 자동문자|
|                   보관함|
|매장 찾기 및 행복 AS 안내|
|         데이터/통화 관리|
|                    PLANS|
|           자주 하는 질문|
|                나의 활동|
|                 공지사항|
|                     My T|
|                   T 로밍|
|         서비스 장애 안내|
+-------------------------+
only showing top 20 rows



In [13]:
data.select('item').distinct().show()

+--------------------------------+
|                            item|
+--------------------------------+
|                서비스 신청 현황|
|                             LTE|
|                     최근 사용량|
|  인터넷/전화/IPTV 이용계약 정보|
|                        결합상품|
|               부가서비스 리스트|
|                       상담 예약|
|      인터넷/전화/B tv 가입 내역|
|                       로밍이란?|
|                T world Site Map|
|                휴대폰 결제 내역|
|             검색 결과 노출 영역|
|기가라이트인터넷 와이파이 + B...|
|     이용자 피해예방 관련 사이트|
|       데이터 함께쓰기 요금 충전|
|                 사이트 이용방법|
|               잔여기본통화 조회|
|                            예약|
|                  칭찬 등록 내역|
|                          LTE/3G|
+--------------------------------+
only showing top 20 rows



In [6]:
def prefer_list(data, cate="cat1", thre=0.7):
    data = data.select(['svc_mgmt_num',f'{cate}']).na.drop("any", subset=f'{cate}')
    data = data.groupby('svc_mgmt_num',f'{cate}').agg(count("*").alias(f'{cate}_cnt'))
    data_user = data.groupby('svc_mgmt_num').agg(max(f'{cate}_cnt').alias(f'{cate}_max_cnt'))
    data = data.join(data_user, on='svc_mgmt_num', how="left")
    data = data.withColumn(f'{cate}_prop', col(f'{cate}_cnt')/col(f'{cate}_max_cnt'))
    data = data.filter(col(f'{cate}_prop') > thre)
    
    window_spec = Window.partitionBy("svc_mgmt_num").orderBy(desc(f'{cate}_prop'))

    top_keywords = data.withColumn("rank", row_number().over(window_spec)) \
                    .filter(col("rank") <= 10) \
                    .drop("rank")
    data = top_keywords.groupBy("svc_mgmt_num").agg(concat_ws(", ",collect_list(f'{cate}')).alias(f'{cate}_list'))
    
    return data

## xtr item level

In [7]:
data_item = data.filter(col('dt') >= f'{DT_threshold2}')
data_item = data_item.select(['svc_mgmt_num','item']).na.drop("any", subset='item')
data_item = prefer_list(data_item, cate="item", thre=0.5)

## xtr cat level
- cat2(더 상위 카테고리) > cat1

- cat1

In [8]:
data_cat1 = prefer_list(data, cate="cat1", thre=0.6)

- cat2

In [9]:
data_cat2 = prefer_list(data, cate="cat2", thre=0.6)

### 전체 데이터 Merge

In [10]:
full_data = data_item.join(data_cat1,on="svc_mgmt_num")
full_data = full_data.join(data_cat2,on="svc_mgmt_num")
full_data = full_data.withColumn("dt", lit(DT).cast("date"))

In [11]:
full_data = full_data.withColumnRenamed("item_list", "xtr_item_prefer")\
                      .withColumnRenamed("cat1_list", "xtr_cat1_prefer")\
                      .withColumnRenamed("cat2_list", "xtr_cat2_prefer")

In [12]:
full_data.printSchema()

root
 |-- svc_mgmt_num: string (nullable = true)
 |-- xtr_item_prefer: string (nullable = false)
 |-- xtr_cat1_prefer: string (nullable = false)
 |-- xtr_cat2_prefer: string (nullable = false)
 |-- dt: date (nullable = true)



### AIDP 저장

In [13]:
dest_dataset = "x1113099"
partitioned_dest_table = "one_model_profile_xtr"

In [14]:
get_bigquery_client().query(f"""
    CREATE TABLE IF NOT EXISTS {dest_dataset}.{partitioned_dest_table}
    (
        svc_mgmt_num STRING,
        xtr_item_prefer STRING,
        xtr_cat1_prefer STRING,
        xtr_cat2_prefer STRING,
        dt DATE,
    )
    PARTITION BY dt
""").result()

print(f"생성된 테이블 : {dest_dataset}.{partitioned_dest_table}")

생성된 테이블 : x1113099.one_model_profile_xtr


In [15]:
df_to_bq_table(df=full_data,
               dataset=dest_dataset,
               table_name=partitioned_dest_table,
               mode="overwrite")